<a href="https://colab.research.google.com/github/Anushka-engg/amazonreview-codtech/blob/main/amazonreview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()  # Select kaggle.json file


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"anushkasarkar11","key":"46eefaf3866675b3f1346a570aacef20"}'}

In [2]:
!pip install kaggle

In [3]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
from google.colab import files
uploaded = files.upload()

Saving Reviews.csv.zip to Reviews.csv.zip


In [5]:
import os
os.listdir()

['.config', 'Reviews.csv.zip', 'sample_data']

In [6]:
from pyspark.sql import SparkSession

# Start Spark session
spark = SparkSession.builder.appName("AmazonReviews").getOrCreate()

# Load dataset into a Spark DataFrame
df_spark = spark.read.csv("snap/amazon-fine-food-reviews", header=True, inferSchema=True)
df_spark.show(5)  # Show first 5 rows

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/content/snap/amazon-fine-food-reviews.

In [7]:
import os
os.listdir()

['.config', 'Reviews.csv.zip', 'sample_data']

In [8]:
import zipfile

with zipfile.ZipFile("Reviews.csv.zip", "r") as zip_ref:
    zip_ref.extractall()  # Extracts in the same directory

# Check extracted files
os.listdir()


['.config', 'Reviews.csv', 'Reviews.csv.zip', 'sample_data']

In [9]:
from pyspark.sql import SparkSession

# Step 1: Start a Spark session
spark = SparkSession.builder.appName("AmazonReviews").getOrCreate()

# Step 2: Load the CSV file into a Spark DataFrame
df_spark = spark.read.csv("Reviews.csv", header=True, inferSchema=True)

# Step 3: Show the first 5 rows
df_spark.show(5)


+---+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+
| Id| ProductId|        UserId|         ProfileName|HelpfulnessNumerator|HelpfulnessDenominator|Score|      Time|             Summary|                Text|
+---+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+
|  1|B001E4KFG0|A3SGXH7AUHU8GW|          delmartian|                   1|                     1|    5|1303862400|Good Quality Dog ...|I have bought sev...|
|  2|B00813GRG4|A1D87F6ZCVE5NK|              dll pa|                   0|                     0|    1|1346976000|   Not as Advertised|"Product arrived ...|
|  3|B000LQOCH0| ABXLMWJIXXAIN|"Natalia Corres "...|                   1|                     1|    4|1219017600|"""Delight"" says...|"This is a confec...|
|  4|B000UA0QIQ|A395BORC6FGVXV|                Karl|            

In [10]:
df_spark.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- ProductId: string (nullable = true)
 |-- UserId: string (nullable = true)
 |-- ProfileName: string (nullable = true)
 |-- HelpfulnessNumerator: string (nullable = true)
 |-- HelpfulnessDenominator: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Summary: string (nullable = true)
 |-- Text: string (nullable = true)



In [11]:
df_spark.count()

568454

In [12]:
df_spark = df_spark.dropna()

In [13]:
df_spark = df_spark.dropDuplicates()

In [15]:
from pyspark.sql.functions import col, to_date


In [16]:
df_spark = df_spark.withColumn("Review_Date", to_date(col("Time").cast("timestamp")))
df_spark.show(5)

+----+----------+--------------+-------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+-----------+
|  Id| ProductId|        UserId|        ProfileName|HelpfulnessNumerator|HelpfulnessDenominator|Score|      Time|             Summary|                Text|Review_Date|
+----+----------+--------------+-------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+-----------+
| 569|B000G6RYNE| AJWLSDYCSQNBL|     Imelda'sFetish|                   2|                     2|    1|1325894400|Kettle Brand Pota...|Kettle Branch Pot...|       NULL|
| 694|B000G6MBX2|A3QAZDJ7NYH2C6|          M. Esmail|                   0|                     0|    5|1265500800|Huge fan of these...|I Love these chip...|       NULL|
| 956|B000S806VM| AIR7BNZFT751N|   Paul K. Williams|                   2|                     2|    4|1251849600|    Good tasten tea.|I ordered this on...|     

In [18]:
df_spark.groupBy("ProductId").count().orderBy("count", ascending=False).show(10)


+----------+-----+
| ProductId|count|
+----------+-----+
|B007JFMH8M|  913|
|B002QWHJOU|  632|
|B002QWP89S|  632|
|B0026RQTGE|  632|
|B002QWP8H0|  632|
|B003B3OOPA|  623|
|B001EO5Q64|  567|
|B007M832YY|  564|
|B001RVFERK|  564|
|B0026KNQSA|  564|
+----------+-----+
only showing top 10 rows



In [17]:
df_spark.groupBy("Score").count().orderBy("Score", ascending=False).show()


+---------------+------+
|          Score| count|
+---------------+------+
|book-blogger"""|    59|
|              9|    22|
|              8|    12|
|              7|    41|
|             69|     3|
|             65|     2|
|             60|     1|
|              6|    38|
|             57|     1|
|             54|     1|
|             51|     2|
|              5|361648|
|             49|     1|
|             47|    14|
|             46|     1|
|             40|     1|
|              4| 80141|
|             35|     1|
|             33|     6|
|             32|     2|
+---------------+------+
only showing top 20 rows



In [19]:
from pyspark.sql.functions import length, avg

df_spark = df_spark.withColumn("Review_Length", length(col("Text")))
df_spark.select(avg("Review_Length")).show()

+------------------+
|avg(Review_Length)|
+------------------+
|375.86301377092553|
+------------------+



In [20]:
from pyspark.sql.functions import when

df_spark = df_spark.withColumn(
    "Sentiment",
    when(df_spark["Score"] >= 4, "Positive")
    .when(df_spark["Score"] == 3, "Neutral")
    .otherwise("Negative")
)

df_spark.groupBy("Sentiment").count().show()

+---------+------+
|Sentiment| count|
+---------+------+
| Positive|442037|
|  Neutral| 42502|
| Negative| 83905|
+---------+------+



In [21]:
df_spark.write.csv("Cleaned_Amazon_Reviews.csv", header=True)

In [22]:
df_spark.describe().show()


AnalysisException: [DATATYPE_MISMATCH.UNEXPECTED_INPUT_TYPE] Cannot resolve "element_at(Summary, summary)" due to data type mismatch: Parameter 1 requires the ("ARRAY" or "MAP") type, however "Summary" has the type "STRING".;
'Project [summary#674, element_at(Id#373, summary#674, None, false) AS Id#429, element_at(ProductId#378, summary#674, None, false) AS ProductId#430, element_at(UserId#383, summary#674, None, false) AS UserId#431, element_at(ProfileName#388, summary#674, None, false) AS ProfileName#432, element_at(HelpfulnessNumerator#393, summary#674, None, false) AS HelpfulnessNumerator#433, element_at(HelpfulnessDenominator#398, summary#674, None, false) AS HelpfulnessDenominator#434, element_at(Score#403, summary#674, None, false) AS Score#435, element_at(Time#408, summary#674, None, false) AS Time#436, element_at(Summary#674, summary#674, None, false) AS Summary#437, element_at(Text#418, summary#674, None, false) AS Text#438, element_at(Review_Length#423, summary#674, None, fa

In [23]:
df_spark.groupBy("ProductId").count().orderBy("count", ascending=False).show(10)


+----------+-----+
| ProductId|count|
+----------+-----+
|B007JFMH8M|  913|
|B002QWHJOU|  632|
|B002QWP89S|  632|
|B0026RQTGE|  632|
|B002QWP8H0|  632|
|B003B3OOPA|  623|
|B001EO5Q64|  567|
|B007M832YY|  564|
|B001RVFERK|  564|
|B0026KNQSA|  564|
+----------+-----+
only showing top 10 rows



In [24]:
df_spark.groupBy("UserId").count().orderBy("count", ascending=False).show(10)


+--------------+-----+
|        UserId|count|
+--------------+-----+
|A3OXHLG6DIBRW8|  448|
|A1YUL9PCJR3JTY|  421|
| AY12DBB0U420B|  389|
|A281NPSIMI1C2R|  365|
|A1Z54EM24Y40LL|  256|
|A1TMAVN4CEM8U8|  204|
|A2MUGFV2TDQ47K|  201|
|A3TVZM3ZIXG8YW|  199|
|A3PJZ8TU8FDQ1K|  178|
| AQQLWCMRNDFGI|  176|
+--------------+-----+
only showing top 10 rows



In [25]:
from pyspark.sql.functions import round

df_spark.groupBy("ProductId").agg(round(avg("Score"),2).alias("Avg_Rating")).orderBy("Avg_Rating", ascending=False).show(10)


+----------+----------+
| ProductId|Avg_Rating|
+----------+----------+
|B000X90P5I|      25.0|
|B004HN3ONQ|     18.67|
|B001EPPGXG|      16.0|
|B003GUTELM|      14.5|
|B0001WYNDM|      8.33|
|B001E530FM|       8.3|
|B001E530FW|       8.3|
|B001SAQCNA|      8.25|
|B000EF3FR6|      7.43|
|B001R7VT7S|      7.06|
+----------+----------+
only showing top 10 rows



In [26]:
from pyspark.sql.functions import to_date, year, month

df_spark = df_spark.withColumn("Review_Date", to_date(col("Time").cast("timestamp")))
df_spark = df_spark.withColumn("Year", year(col("Review_Date")))
df_spark = df_spark.withColumn("Month", month(col("Review_Date")))


In [27]:
df_spark.groupBy("Year").count().orderBy("Year").show()


+----+------+
|Year| count|
+----+------+
|NULL|568444|
+----+------+



In [28]:
df_spark.groupBy("Year", "Month").count().orderBy("Year", "Month").show()


+----+-----+------+
|Year|Month| count|
+----+-----+------+
|NULL| NULL|568444|
+----+-----+------+



In [29]:
from pyspark.sql.functions import split, explode

df_words = df_spark.withColumn("word", explode(split(col("Text"), " ")))
df_words.groupBy("word").count().orderBy("count", ascending=False).show(20)


+----+-------+
|word|  count|
+----+-------+
| the|1399375|
|   I|1223941|
| and|1075449|
|   a|1011472|
|    | 876643|
|  to| 858001|
|  of| 669140|
|  is| 625951|
|  it| 555091|
| for| 456043|
|  in| 443542|
|this| 433784|
|that| 339127|
|  my| 324922|
|have| 298911|
|with| 290234|
| but| 279388|
| was| 277122|
| are| 272878|
| not| 247113|
+----+-------+
only showing top 20 rows



In [30]:
from pyspark.sql.functions import when

df_spark = df_spark.withColumn(
    "Sentiment",
    when(df_spark["Score"] >= 4, "Positive")
    .when(df_spark["Score"] == 3, "Neutral")
    .otherwise("Negative")
)

df_spark.groupBy("Sentiment").count().show()


+---------+------+
|Sentiment| count|
+---------+------+
| Positive|442037|
|  Neutral| 42502|
| Negative| 83905|
+---------+------+



In [31]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [32]:
df = df.select("Text", "Score").dropna()
df = df.withColumn("Score", df["Score"].cast("integer"))

# Convert ratings to Sentiment Categories
df = df.withColumn(
    "Sentiment",
    when(df["Score"] >= 4, "Positive")
    .when(df["Score"] == 3, "Neutral")
    .otherwise("Negative")
)

df.show(5)

NameError: name 'df' is not defined

In [33]:
df = df.select("Text", "Score").dropna()
df = df.withColumn("Score", df["Score"].cast("integer"))

# Convert ratings to Sentiment Categories
df = df.withColumn(
    "Sentiment",
    when(df["Score"] >= 4, "Positive")
    .when(df["Score"] == 3, "Neutral")
    .otherwise("Negative")
)

df.show(5)

NameError: name 'df' is not defined

In [34]:
tokenizer = Tokenizer(inputCol="Text", outputCol="words")
df_tokens = tokenizer.transform(df)
df_tokens.show(5)


NameError: name 'df' is not defined

In [35]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("AmazonReviews").getOrCreate()

# Load the dataset
df = spark.read.csv("Reviews.csv", header=True, inferSchema=True)

# Show data
df.show(5)


+---+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+
| Id| ProductId|        UserId|         ProfileName|HelpfulnessNumerator|HelpfulnessDenominator|Score|      Time|             Summary|                Text|
+---+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+
|  1|B001E4KFG0|A3SGXH7AUHU8GW|          delmartian|                   1|                     1|    5|1303862400|Good Quality Dog ...|I have bought sev...|
|  2|B00813GRG4|A1D87F6ZCVE5NK|              dll pa|                   0|                     0|    1|1346976000|   Not as Advertised|"Product arrived ...|
|  3|B000LQOCH0| ABXLMWJIXXAIN|"Natalia Corres "...|                   1|                     1|    4|1219017600|"""Delight"" says...|"This is a confec...|
|  4|B000UA0QIQ|A395BORC6FGVXV|                Karl|            

In [36]:
df = df.select("Text", "Score").dropna()
df = df.withColumn("Score", df["Score"].cast("integer"))

# Convert ratings to Sentiment Categories
df = df.withColumn(
    "Sentiment",
    when(df["Score"] >= 4, "Positive")
    .when(df["Score"] == 3, "Neutral")
    .otherwise("Negative")
)

df.show(5)


+--------------------+-----+---------+
|                Text|Score|Sentiment|
+--------------------+-----+---------+
|I have bought sev...|    5| Positive|
|"Product arrived ...|    1| Negative|
|"This is a confec...|    4| Positive|
|If you are lookin...|    2| Negative|
|Great taffy at a ...|    5| Positive|
+--------------------+-----+---------+
only showing top 5 rows



In [37]:
tokenizer = Tokenizer(inputCol="Text", outputCol="words")
df_tokens = tokenizer.transform(df)
df_tokens.show(5)


+--------------------+-----+---------+--------------------+
|                Text|Score|Sentiment|               words|
+--------------------+-----+---------+--------------------+
|I have bought sev...|    5| Positive|[i, have, bought,...|
|"Product arrived ...|    1| Negative|["product, arrive...|
|"This is a confec...|    4| Positive|["this, is, a, co...|
|If you are lookin...|    2| Negative|[if, you, are, lo...|
|Great taffy at a ...|    5| Positive|[great, taffy, at...|
+--------------------+-----+---------+--------------------+
only showing top 5 rows



In [38]:
stopwords_remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
df_clean = stopwords_remover.transform(df_tokens)
df_clean.select("filtered_words").show(10, truncate=False)


+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|filtered_words                                                                                                                                                                                                                                                                                                                             |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [39]:
tokenizer = Tokenizer(inputCol="Text", outputCol="words")
df_tokens = tokenizer.transform(df)
df_tokens.show(5)

+--------------------+-----+---------+--------------------+
|                Text|Score|Sentiment|               words|
+--------------------+-----+---------+--------------------+
|I have bought sev...|    5| Positive|[i, have, bought,...|
|"Product arrived ...|    1| Negative|["product, arrive...|
|"This is a confec...|    4| Positive|["this, is, a, co...|
|If you are lookin...|    2| Negative|[if, you, are, lo...|
|Great taffy at a ...|    5| Positive|[great, taffy, at...|
+--------------------+-----+---------+--------------------+
only showing top 5 rows



In [40]:
hashingTF = HashingTF(inputCol="filtered_words", outputCol="raw_features", numFeatures=1000)
df_hashed = hashingTF.transform(df_clean)

idf = IDF(inputCol="raw_features", outputCol="features")
idf_model = idf.fit(df_hashed)
df_features = idf_model.transform(df_hashed)

df_features.select("features").show(5)


+--------------------+
|            features|
+--------------------+
|(1000,[61,78,168,...|
|(1000,[42,104,121...|
|(1000,[66,116,143...|
|(1000,[10,61,73,2...|
|(1000,[26,54,128,...|
+--------------------+
only showing top 5 rows



In [41]:
lr = LogisticRegression(featuresCol="features", labelCol="Sentiment_Label")
model = lr.fit(train_data)

NameError: name 'train_data' is not defined

In [42]:
# Split the dataset into training (80%) and testing (20%)
train_data, test_data = df_final.randomSplit([0.8, 0.2], seed=42)


NameError: name 'df_final' is not defined

In [43]:
%who_ls



['HashingTF',
 'IDF',
 'LogisticRegression',
 'MulticlassClassificationEvaluator',
 'SparkSession',
 'StopWordsRemover',
 'StringIndexer',
 'Tokenizer',
 'VectorAssembler',
 'avg',
 'col',
 'df',
 'df_clean',
 'df_features',
 'df_hashed',
 'df_spark',
 'df_tokens',
 'df_words',
 'explode',
 'files',
 'hashingTF',
 'idf',
 'idf_model',
 'length',
 'lr',
 'month',
 'os',
 'pyspark',
 'round',
 'spark',
 'split',
 'stopwords_remover',
 'to_date',
 'tokenizer',
 'uploaded',
 'when',
 'year',
 'zip_ref',
 'zipfile']

In [44]:
# Rename df to df_final
df_final = df


In [45]:
train_data, test_data = df_final.randomSplit([0.8, 0.2], seed=42)


In [46]:
df_clean.printSchema()
df_spark.printSchema()


root
 |-- Text: string (nullable = true)
 |-- Score: integer (nullable = true)
 |-- Sentiment: string (nullable = false)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- filtered_words: array (nullable = true)
 |    |-- element: string (containsNull = true)

root
 |-- Id: integer (nullable = true)
 |-- ProductId: string (nullable = true)
 |-- UserId: string (nullable = true)
 |-- ProfileName: string (nullable = true)
 |-- HelpfulnessNumerator: string (nullable = true)
 |-- HelpfulnessDenominator: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Summary: string (nullable = true)
 |-- Text: string (nullable = true)
 |-- Review_Date: date (nullable = true)
 |-- Review_Length: integer (nullable = true)
 |-- Sentiment: string (nullable = false)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)



In [47]:
df_final = df_clean  # or df_spark


In [48]:
train_data, test_data = df_final.randomSplit([0.8, 0.2], seed=42)


In [49]:
lr = LogisticRegression(featuresCol="features", labelCol="Sentiment_Label")
model = lr.fit(train_data)

IllegalArgumentException: features does not exist. Available: Text, Score, Sentiment, words, filtered_words

In [50]:
train_data, test_data = df_final.randomSplit([0.8, 0.2], seed=42)

In [51]:
lr = LogisticRegression(featuresCol="features", labelCol="Sentiment_Label")
model = lr.fit(train_data)

IllegalArgumentException: features does not exist. Available: Text, Score, Sentiment, words, filtered_words

In [52]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover, VectorAssembler

# Tokenize the text
tokenizer = Tokenizer(inputCol="Text", outputCol="words")
df_tokens = tokenizer.transform(df)

# Remove stopwords
stopwords_remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
df_clean = stopwords_remover.transform(df_tokens)

# Apply HashingTF
hashingTF = HashingTF(inputCol="filtered_words", outputCol="raw_features", numFeatures=1000)
df_hashed = hashingTF.transform(df_clean)

# Compute IDF to get final feature vector
idf = IDF(inputCol="raw_features", outputCol="features")
idf_model = idf.fit(df_hashed)
df_features = idf_model.transform(df_hashed)

# Show the new dataframe
df_features.select("Text", "features").show(5, truncate=False)


+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [53]:
train_data, test_data = df_final.randomSplit([0.8, 0.2], seed=42)

# Train Logistic Regression Model
lr = LogisticRegression(featuresCol="features", labelCol="Sentiment_Label")
model = lr.fit(train_data)

# Make Predictions
predictions = model.transform(test_data)
predictions.select("Sentiment_Label", "prediction").show(10)

# Evaluate Model
evaluator = MulticlassClassificationEvaluator(labelCol="Sentiment_Label", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Model Accuracy: {accuracy:.2f}")


IllegalArgumentException: features does not exist. Available: Text, Score, Sentiment, words, filtered_words

In [54]:
from pyspark.ml.feature import Tokenizer

tokenizer = Tokenizer(inputCol="Text", outputCol="words")
df_tokens = tokenizer.transform(df)


In [55]:
from pyspark.ml.feature import StopWordsRemover

stopwords_remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
df_clean = stopwords_remover.transform(df_tokens)


In [56]:
from pyspark.ml.feature import HashingTF, IDF

hashingTF = HashingTF(inputCol="filtered_words", outputCol="raw_features", numFeatures=1000)
df_hashed = hashingTF.transform(df_clean)

idf = IDF(inputCol="raw_features", outputCol="features")
idf_model = idf.fit(df_hashed)
df_features = idf_model.transform(df_hashed)


In [57]:
train_data, test_data = df_final.randomSplit([0.8, 0.2], seed=42)


In [59]:
predictions = model.transform(test_data)
predictions.select("Sentiment_Label", "prediction").show(10)

NameError: name 'model' is not defined

In [60]:
print(model)


NameError: name 'model' is not defined

In [61]:
print("Available variables:", locals().keys())


Available variables: dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 'files', '_1', '_i2', '_exit_code', '_i3', '_i4', 'uploaded', '_i5', 'os', '_5', '_i6', 'SparkSession', 'spark', '_i7', '_7', '_i8', 'zipfile', 'zip_ref', '_8', '_i9', 'df_spark', '_i10', '_i11', '_11', '_i12', '_i13', '_i14', 'to_date', '_i15', 'col', '_i16', '_i17', '_i18', '_i19', 'length', 'avg', '_i20', 'when', '_i21', '_i22', '_i23', '_i24', '_i25', 'round', '_i26', 'year', 'month', '_i27', '_i28', '_i29', 'split', 'explode', 'df_words', '_i30', '_i31', 'pyspark', 'Tokenizer', 'StopWordsRemover', 'HashingTF', 'IDF', 'StringIndexer', 'VectorAssembler', 'LogisticRegression', 'MulticlassClassificationEvaluator', '_i32', '_i33', '_i34', 'tokenizer', '_i35', 'df', '_i36', '_i37', 'df_tokens', '_i38', 'stopwords_remover', 'df_clean', '_i39', '_i40', 'h

In [62]:
print(df_final.columns)  # Ensure "features" is present


['Text', 'Score', 'Sentiment', 'words', 'filtered_words']


In [63]:
from pyspark.ml.feature import StringIndexer

# Convert sentiment labels into numerical values
indexer = StringIndexer(inputCol="Sentiment", outputCol="Sentiment_Label")
df_final = indexer.fit(df_final).transform(df_final)

# Verify the new column
df_final.select("Sentiment", "Sentiment_Label").show(5)


+---------+---------------+
|Sentiment|Sentiment_Label|
+---------+---------------+
| Positive|            0.0|
| Negative|            1.0|
| Positive|            0.0|
| Negative|            1.0|
| Positive|            0.0|
+---------+---------------+
only showing top 5 rows



In [64]:
predictions = model.transform(test_data)
predictions.select("Sentiment_Label", "prediction").show(10)

NameError: name 'model' is not defined

In [65]:
from pyspark.ml.feature import HashingTF, IDF, VectorAssembler

# Convert words into numerical representation
hashingTF = HashingTF(inputCol="filtered_words", outputCol="raw_features", numFeatures=1000)
df_hashed = hashingTF.transform(df_final)

idf = IDF(inputCol="raw_features", outputCol="features")
idf_model = idf.fit(df_hashed)
df_features = idf_model.transform(df_hashed)

# Now df_features contains the required 'features' column
df_final = df_features.select("features", "Sentiment_Label")

# Verify columns
df_final.show(5)


+--------------------+---------------+
|            features|Sentiment_Label|
+--------------------+---------------+
|(1000,[61,78,168,...|            0.0|
|(1000,[42,104,121...|            1.0|
|(1000,[66,116,143...|            0.0|
|(1000,[10,61,73,2...|            1.0|
|(1000,[26,54,128,...|            0.0|
+--------------------+---------------+
only showing top 5 rows



In [69]:
df_final.coalesce(1).write.csv("amazon_reviews_cleaned", header=True, mode="overwrite")


AnalysisException: [UNSUPPORTED_DATA_TYPE_FOR_DATASOURCE] The CSV datasource doesn't support the column `features` of the type "STRUCT<type: TINYINT, size: INT, indices: ARRAY<INT>, values: ARRAY<DOUBLE>>".

In [70]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType


In [71]:
# Define UDF (User Defined Function) to convert Vector to String
vector_to_string = udf(lambda vec: ",".join([str(x) for x in vec.toArray()]), StringType())

# Apply transformation
df_final = df_final.withColumn("features_str", vector_to_string(df_final["features"]))

# Drop old 'features' column
df_final = df_final.drop("features")

# Show data before saving
df_final.show(5)


+---------------+--------------------+
|Sentiment_Label|        features_str|
+---------------+--------------------+
|            0.0|0.0,0.0,0.0,0.0,0...|
|            1.0|0.0,0.0,0.0,0.0,0...|
|            0.0|0.0,0.0,0.0,0.0,0...|
|            1.0|0.0,0.0,0.0,0.0,0...|
|            0.0|0.0,0.0,0.0,0.0,0...|
+---------------+--------------------+
only showing top 5 rows



In [72]:
df_final.coalesce(1).write.csv("amazon_reviews_cleaned", header=True, mode="overwrite")



In [73]:
df_final.coalesce(1).write.csv("amazon_reviews_cleaned", header=True, mode="overwrite")
